# 5 Layer Convnet for Classifying 5  Types of Flowers

In [81]:
# import necessary dependencies and files
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from features import Features
from tqdm import tqdm
import os, sys
%matplotlib inline

### Load in the datasets

In [82]:
features = Features()
if not os.path.isfile('datasets.npy'):
    datasets = features.create(flatten=False)
else:
    datasets = np.load('datasets.npy')
X_train, y_train, X_test, y_test = features.train_test_split(datasets)

100%|██████████| 15/15 [00:00<00:00, 452.23it/s]


In [87]:
datasets[0][0].shape

(50, 50)

In [83]:
print('Length of training set: {:,}'.format(len(y_train)))
print('Length of testing set:  {:,}'.format(len(y_test)))

Length of training set: 135
Length of testing set:  14


In [84]:
print('X_train.shape = {} \t X_test.shape = {}'.format(X_train.shape, X_test.shape))
print('y_train.shape = {} \t y_test.shape = {}'.format(y_train.shape, y_test.shape))

X_train.shape = (135, 50, 50) 	 X_test.shape = (14, 50, 50)
y_train.shape = (135, 10) 	 y_test.shape = (14, 10)


In [85]:
X_train[0]

array([[ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       ..., 
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  213.,  207.,  204.]], dtype=float32)

In [45]:
y_train[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.], dtype=float32)

In [46]:
X_test[0]

array([[ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       ..., 
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  229.],
       [ 255.,  255.,  253., ...,  244.,  252.,  255.]], dtype=float32)

In [47]:
y_test[0]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

### Define Hyperparameters

In [48]:
# Image & labels
image_size = features.image_size
image_channel = 3
image_shape = (image_size, image_size, image_channel)
image_shape_flat = image_size * image_size * image_channel = 3
num_classes = len(features.classes)

# Network
filter_size = 5
hidden1_channels = 8
hidden2_channels = 16
hidden3_channels = 32
hidden4_channels = 64
hidden5_channels = 128
fully_connected_1 = 512
fully_connected_2 = 256


# Training
learning_rate = 1e-3
dropout = 0.8
iterations = 0
batch_size = 25

### Helper functions for `weights`, `biases`, `conv2d`, & `max_pool`

In [49]:
# Weight initialization
def weight(shape):
    initial = tf.truncated_normal(shape=shape, stddev=1.0/np.sqrt(shape[-2]))
    return tf.Variable(initial)

# Bias in initialization
def bias(length):
    initial = tf.zeros(shape=[length])
    return tf.Variable(initial)

# Convolutional operation
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')

# Max pooling operation
def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [50]:
# Placeholder Variables
X = tf.placeholder(tf.float32, [None, image_shape_flat], name='X_input')
X_image = tf.reshape(X, [None, image_size, image_size, num_channel])
y = tf.placeholder(tf.float32, [None, num_classes], name='Y_input')
keep_prob = tf.placeholder(tf.float32)
y_true = tf.argmax(y, axis=1)

In [51]:
y

<tf.Tensor 'Y_input_1:0' shape=(?, 10) dtype=float32>

In [52]:
y_true

<tf.Tensor 'ArgMax_2:0' shape=(?,) dtype=int64>

## Building the Network
### (1st Convolutional Layer + Max pooling) Input Layer `>` Hidden Layer 1

In [53]:
X_image = tf.reshape(X, shape=[-1, image_size, image_size, num_channel])

In [54]:
W_hidden1 = weight(shape=[filter_size, filter_size, num_channel, hidden1_channels])
b_hidden1 = bias(length=hidden1_channels)
h_conv1 = tf.nn.relu(conv2d(X_image, W_hidden1) + b_hidden1)
h_pool1 = max_pool(h_conv1)

In [55]:
h_pool1

<tf.Tensor 'MaxPool_4:0' shape=(?, 25, 25, 8) dtype=float32>

### (2nd Convolutional Layer + Max pooling) Hidden Layer 1 `>` Hidden Layer 2

In [56]:
W_hidden2 = weight(shape=[filter_size, filter_size, hidden1_channels, hidden2_channels])
b_hidden2 = bias(length=hidden2_channels)
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_hidden2) + b_hidden2)
h_pool2 = max_pool(h_conv2)

In [57]:
h_pool2

<tf.Tensor 'MaxPool_5:0' shape=(?, 13, 13, 16) dtype=float32>

### (3rd Convolutional Layer + Max pooling) Hidden Layer 2 `>` Hidden Layer 3

In [58]:
W_hidden3 = weight(shape=[filter_size, filter_size, hidden2_channels, hidden3_channels])
b_hidden3 = bias(length=hidden3_channels)
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_hidden3) + b_hidden3)
h_pool3 = max_pool(h_conv3)

In [59]:
h_pool3

<tf.Tensor 'MaxPool_6:0' shape=(?, 7, 7, 32) dtype=float32>

### (4th Convolutional Layer + Max pooling) Hidden Layer 3 `>` Hidden Layer 4

In [60]:
W_hidden4 = weight(shape=[filter_size, filter_size, hidden3_channels, hidden4_channels])
b_hidden4 = bias(length=hidden4_channels)
h_conv4 = tf.nn.relu(conv2d(h_conv3, W_hidden4) + b_hidden4)
h_pool4 = max_pool(h_conv4)

In [61]:
h_pool4

<tf.Tensor 'MaxPool_7:0' shape=(?, 7, 7, 64) dtype=float32>

### (5th Convolutional Layer) Hidden Layer 4 `>` Hidden Layer 5

In [62]:
W_hidden5 = weight(shape=[filter_size, filter_size, hidden4_channels, hidden5_channels])
b_hidden5 = bias(length=hidden5_channels)
h_conv5 = tf.nn.relu(conv2d(h_pool4, W_hidden5) + b_hidden5)

In [63]:
h_conv5

<tf.Tensor 'Relu_11:0' shape=(?, 7, 7, 128) dtype=float32>

### (1st Fully Connected Layer) Hidden Layer 5 `>` Hidden Layer 6

In [64]:
h_conv5_flat = tf.reshape(h_conv5, shape=[-1, 19*19*hidden5_channels])
W_fc1 = weight(shape=[19*19*hidden5_channels, fully_connected_1])
b_fc1 = bias(length=fully_connected_1)
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

In [65]:
h_fc1

<tf.Tensor 'Relu_12:0' shape=(?, 512) dtype=float32>

### (2nd Fully Connected Layer + Dropout) Hidden Layer 6 `>` Hidden Layer 7

In [66]:
W_fc2 = weight(shape=[fully_connected_1, fully_connected_2])
b_fc2 = bias(length=fully_connected_2)
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
h_drop = tf.nn.dropout(h_fc2, keep_prob=keep_prob)

In [67]:
h_drop

<tf.Tensor 'dropout_1/mul:0' shape=(?, 256) dtype=float32>

### Readout/Output Layer

In [68]:
W_out = weight(shape=[fully_connected_2, num_classes])
b_out = bias(length=num_classes)
y_pred = tf.matmul(h_drop, W_out) + b_out
y_pred_true = tf.argmax(y_pred, axis=1)

In [69]:
y_pred

<tf.Tensor 'add_15:0' shape=(?, 10) dtype=float32>

In [70]:
y_pred_true

<tf.Tensor 'ArgMax_3:0' shape=(?,) dtype=int64>

### Cost function, and optimizer

In [71]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y, name='xentropy')
cost = tf.reduce_mean(cross_entropy, name='xentropy_mean')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Evaluating accuracy

In [72]:
correct = tf.equal(y_true, y_pred_true)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Running tensorflow's `Session()`

In [73]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

### `optimize()` and `print_accuracy()` function helper

In [74]:
# feed_dict_train = {X: X_train, 
#                    y: y_train, 
#                    keep_prob:dropout}
feed_dict_test = {X: X_test, 
                  y: y_test, 
                  keep_prob: dropout}

# Optimize helper
def optimize(num_iter=1):
    global iterations
    for i in tqdm(range(num_iter)):
        start = 0
        while start < len(X_train):
            end = start + batch_size
            batch_X = X_train[start:end]
            batch_y = y_train[start:end]
            sess.run(optimizer, feed_dict={X:batch_X,
                                           y:batch_y,
                                           keep_prob:dropout})
            start += batch_size
        iterations += 1


# Accuracy helper
def print_accuracy():
    acc = sess.run(accuracy, feed_dict=feed_dict_test)
    print('Accuracy after {:,} iterations = {:.2%}'.format(iterations, acc))

## Training the network

In [75]:
print_accuracy()

ValueError: Cannot feed value of shape (14, 50, 50) for Tensor 'X_input_1:0', which has shape '(?, 7500)'

In [76]:
optimize() # for 1 iteration

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Cannot feed value of shape (25, 50, 50) for Tensor 'X_input_1:0', which has shape '(?, 7500)'

In [ ]:
print_accuracy()

Accuracy after 1 iterations = 21.53%


In [ ]:
optimize(9) # to complete 10 iterations

 33%|███▎      | 3/9 [10:43<21:26, 214.45s/it]

In [ ]:
print_accuracy()

In [ ]:
optimize(90) # to complete 100 iterations

In [ ]:
print_accuracy()

In [ ]:
optimize(900) # to complete 1,000 iterations

### Closing the tesorflow's `Session()`

In [ ]:
sess.close()